In [1]:
os.chdir('/Users/akhileshvarmabhupathiraju/Documents/taxi_ml_project/NYC_taxi_predictor/')


In [2]:
# import sys
# sys.path.insert(0, '/Users/akhileshvarmabhupathiraju/Documents/taxi_ml_project/NYC_taxi_predictor/')

# from src import paths

In [2]:
# HOPSWORKS_PROJECT_NAME = 'taxi_demand'
HOPSWORKS_PROJECT_NAME = "ny_taxi_wala"

In [3]:
os.environ["HOPWORKS_API_KEY"]

'5deYOvFfMwkw18i2.Qs8uRwvKAtWhRVwlZC5IpRoLNhOO7nOrqH10HrFIL8rKQbqD7pdQ9Bj9Ry2FYH5e'

In [4]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ["HOPWORKS_API_KEY"]


In [5]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in the local storage
File 2022-02 was already in the local storage
File 2022-03 was already in the local storage
File 2022-04 was already in the local storage
File 2022-05 was already in the local storage
File 2022-06 was already in the local storage
File 2022-07 was already in the local storage
File 2022-08 was already in the local storage
File 2022-09 was already in the local storage
File 2022-10 was already in the local storage
File 2022-11 was already in the local storage
File 2022-12 was already in the local storage
File 2023-01 was already in the local storage
File 2023-02 was already in the local storage
File 2023-03 was already in the local storage
2023-04 file is not available
2023-05 file is not available
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [6]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

  pickup_hour  pickup_location_id   0
0  2022-01-01                   4  11
1  2022-01-01                   7   6
2  2022-01-01                  10   1
3  2022-01-01                  12   2
4  2022-01-01                  13  12
Index(['pickup_hour', 'pickup_location_id', 0], dtype='object')
Index(['pickup_hour', 'pickup_location_id', 'rides'], dtype='object')
(1083794, 3)


In [7]:
import hopsworks

In [8]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/49352


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [11]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/2861040 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/49352/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x28a17fb20>, None)